<a href="https://colab.research.google.com/github/lorenzozanisi/recommenderSystems/blob/master/NeuMF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import tensorflow as tf
import tensorflow.keras.layers as Layers
from tensorflow.keras.layers.experimental.preprocessing import CategoryEncoding
from tensorflow.keras.models import Model

In [15]:
!wget 'http://files.grouplens.org/datasets/movielens/ml-100k.zip'

--2021-02-26 17:47:10--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  14.5MB/s    in 0.3s    

2021-02-26 17:47:11 (14.5 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]



In [16]:
!unzip ml-100k.zip

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-100k/ub.test         


In [21]:
!cat ml-100k/README

SUMMARY & USAGE LICENSE

MovieLens data sets were collected by the GroupLens Research Project
at the University of Minnesota.
 
This data set consists of:
	* 100,000 ratings (1-5) from 943 users on 1682 movies. 
	* Each user has rated at least 20 movies. 
        * Simple demographic info for the users (age, gender, occupation, zip)

The data was collected through the MovieLens web site
(movielens.umn.edu) during the seven-month period from September 19th, 
1997 through April 22nd, 1998. This data has been cleaned up - users
who had less than 20 ratings or did not have complete demographic
information were removed from this data set. Detailed descriptions of
the data file can be found at the end of this file.

Neither the University of Minnesota nor any of the researchers
involved can guarantee the correctness of the data, its suitability
for any particular purpose, or the validity of results based on the
use of the data set.  The data set may be used for any research
purposes under th

In [65]:
import pandas as pd
data = pd.read_csv('ml-100k/u.data', sep='\t', names=['userID','itemID','rating','timestamp'])
data = data.drop(columns='timestamp')

In [66]:
data

,userID,itemID,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


In [69]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(data,test_size=0.2)
trainset = tf.data.Dataset.from_tensor_slices(dict(train))


In [72]:
# adapted from https://keras.io/examples/structured_data/structured_data_classification_from_scratch/
def encode_integer_categorical_feature(feature, name, dataset):
    # Create a CategoryEncoding for our integer indices
    encoder = CategoryEncoding(output_mode="binary")

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x: tf.expand_dims(x[name],-1))

    # Learn the space of possible indices
    encoder.adapt(feature_ds)

    # Apply one-hot encoding to our indices
    encoded_feature = encoder(feature)
    return encoded_feature


In [98]:
#preprocess withone hot encoding
user = Layers.Input(shape=(1,))
item = Layers.Input(shape=(1,))
user_encoded = encode_integer_categorical_feature(user, "userID", trainset)
item_encoded = encode_integer_categorical_feature(item, "itemID", trainset)



In [99]:
MF_embedding_size = 10
MLP_embedding_size = 15


#GMF branch

embed_MF_user = Layers.Dense(MF_embedding_size)(user_encoded)
embed_MF_item = Layers.Dense(MF_embedding_size)(item_encoded)
GMF = Layers.Dot(axes=(1))([embed_MF_user, embed_MF_item])


Dense_1_user = Layers.Dense(MLP_embedding_size)(user_encoded)
Dense_1_item = Layers.Dense(MLP_embedding_size)(item_encoded)

In [110]:
import numpy as np

all_inputs = Layers.concatenate([user_encoded,item_encoded])
m = Model([user,item], GMF)
m([np.ones(22)+10,np.ones(22)])

<tf.Tensor: shape=(22, 1), dtype=float32, numpy=
array([[0.00398839],
       [0.00398839],
       [0.00398839],
       [0.00398839],
       [0.00398839],
       [0.00398839],
       [0.00398839],
       [0.00398839],
       [0.00398839],
       [0.00398839],
       [0.00398839],
       [0.00398839],
       [0.00398839],
       [0.00398839],
       [0.00398839],
       [0.00398839],
       [0.00398839],
       [0.00398839],
       [0.00398839],
       [0.00398839],
       [0.00398839],
       [0.00398839]], dtype=float32)>

In [41]:


trainset = trainset.map(
  lambda x, y: (encoder(x), y))
